# FileSystem (Polyglot)

In [ ]:
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.1.5/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"

In [ ]:
#!import ../../lib/fsharp/Common.fs
#!import ../../lib/fsharp/CommonFSharp.fs
#!import ../../lib/fsharp/Async.fs
#!import ../../lib/fsharp/AsyncSeq.fs
#!import ../../lib/fsharp/Runtime.fs

In [ ]:
open Common

## Operators

In [ ]:
module Operators =
    let inline (</>) a b =
        System.IO.Path.Combine (a, b)

In [ ]:
open Operators

## createTempDirectoryName

In [ ]:
let inline createTempDirectoryName () =
    let root = System.Reflection.Assembly.GetEntryAssembly().GetName().Name

    System.IO.Path.GetTempPath ()
    </> $"!{root}"
    </> string (newGuidFromDateTime System.DateTime.Now)

In [ ]:
//// test

createTempDirectoryName ()
|> _contains System.IO.Path.DirectorySeparatorChar

C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231027-1826-2502-0200-019f817de02f


## createTempDirectory

In [ ]:
let inline createTempDirectory () =
    let tempFolder = createTempDirectoryName ()
    let result = System.IO.Directory.CreateDirectory tempFolder

    if not result.Exists then
        let getLocals () =
            $"tempFolder: {tempFolder} / result: {({|
                Exists = result.Exists
                CreationTime = result.CreationTime
            |})} {getLocals ()}"

        trace Debug (fun () -> "createTempDirectory") getLocals

    tempFolder

In [ ]:
//// test

let tempDirectory = createTempDirectory ()

Directory.Exists tempDirectory
|> _equal true

True


## getSourceDirectory

In [ ]:
let getSourceDirectory =
    fun () -> __SOURCE_DIRECTORY__
    |> memoize

In [ ]:
//// test

getSourceDirectory ()
|> System.IO.DirectoryInfo
|> fun dir -> dir.Name
|> _equal "fsharp"

fsharp


## findParent

In [ ]:
let inline findParent name isFile rootDir =
    let rec loop dir =
        if dir </> name |> (if isFile then System.IO.File.Exists else System.IO.Directory.Exists)
        then dir
        else
            dir
            |> System.IO.Directory.GetParent
            |> function
                | null -> failwith $"""No parent for {if isFile then "file" else "dir"} '{name}' at '{rootDir}'"""
                | parent -> parent.FullName |> loop
    loop rootDir

In [ ]:
//// test

getSourceDirectory () |> findParent ".paket" false
|> System.IO.DirectoryInfo
|> fun dir -> dir.Name
|> _equal "polyglot"

polyglot


In [ ]:
//// test

getSourceDirectory () |> findParent "paket.dependencies" true
|> System.IO.DirectoryInfo
|> fun dir -> dir.Name
|> _equal "polyglot"

polyglot


## readAllTextAsync

In [ ]:
let inline readAllTextAsync path =
    path |> System.IO.File.ReadAllTextAsync |> Async.AwaitTask

## fileExistsContent

In [ ]:
let inline fileExistsContent path content = async {
    if path |> System.IO.File.Exists |> not
    then return false
    else
        let! existingContent = path |> readAllTextAsync
        return content = existingContent
}

## writeAllTextAsync

In [ ]:
let inline writeAllTextAsync path contents =
    System.IO.File.WriteAllTextAsync (path, contents) |> Async.AwaitTask

## writeAllTextExists

In [ ]:
let inline writeAllTextExists path contents = async {
    let! exists = contents |> fileExistsContent path
    if not exists
    then do! contents |> writeAllTextAsync path
}

## waitForFileAccess

In [ ]:
let inline waitForFileAccess access path =
    let fileAccess, fileShare =
        access
        |> Option.defaultValue (System.IO.FileAccess.ReadWrite, System.IO.FileShare.Read)

    let rec loop retry = async {
        try
            use _ = new System.IO.FileStream (
                path,
                System.IO.FileMode.Open,
                fileAccess,
                fileShare
            )
            return retry
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path |> System.IO.Path.GetFileName} / ex: {ex |> printException} / {getLocals ()}"
                trace Debug (fun () -> "waitForFileAccess") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"

let inline lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use stream = new System.IO.FileStream (
        path,
        System.IO.FileMode.Open,
        System.IO.FileAccess.ReadWrite,
        System.IO.FileShare.None
    )
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
    stream.Seek (0L, SeekOrigin.Begin) |> ignore
    trace Debug (fun () -> "_4") getLocals
    stream.WriteByte 49uy
    trace Debug (fun () -> "_5") getLocals
    stream.Flush ()
    trace Debug (fun () -> "_6") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! "0" |> writeAllTextAsync path
    trace Debug (fun () -> "2") getLocals
    let! child = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    let! retries = path |> waitForFileAccess None
    trace Debug (fun () -> "5") getLocals
    let! text = path |> readAllTextAsync
    trace Debug (fun () -> "6") getLocals
    do! child
    trace Debug (fun () -> "7") getLocals
    return retries, text
}
|> Async.runWithTimeout 3000
|> function
    | Some (retries, text) ->
        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 100)
            (if Runtime.isWindows () then 150 else 200)
        
        text |> _equal "1"
        
        true
    | _ -> false
|> _equal true

00:00:00 #1 [Debug] 1
00:00:00 #2 [Debug] 2
00:00:00 #3 [Debug] _1
00:00:00 #4 [Debug] 3
00:00:00 #5 [Debug] _2
00:00:00 #6 [Debug] 4
00:00:00 #7 [Debug] waitForFileAccess / path: test.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231027-1826-2581-8151-8493d0e5cebb\test.txt' because it is being used by another process.
00:00:01 #8 [Debug] waitForFileAccess / path: test.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231027-1826-2581-8151-8493d0e5cebb\test.txt' because it is being used by another process.
00:00:02 #9 [Debug] _3
00:00:02 #10 [Debug] _4
00:00:02 #11 [Debug] _5
00:00:02 #12 [Debug] _6
00:00:02 #13 [Debug] 5
00:00:02 #14 [Debug] 6
00:00:02 #15 [Debug] 7
128
128
128
1
True


## readAllTextRetryAsync

In [ ]:
let inline readAllTextRetryAsync fullPath =
    let rec loop retry = async {
        try
            if retry > 0
            then do!
                fullPath
                |> waitForFileAccess (Some (
                    System.IO.FileAccess.Read,
                    System.IO.FileShare.Read
                ))
                |> Async.runWithTimeoutAsync 1000
                |> Async.Ignore
            return! fullPath |> readAllTextAsync |> Async.map Some
        with ex ->
            let getLocals () = $"retry: {retry} / ex: {ex |> printException} / {getLocals ()}"
            trace Debug (fun () -> $"watchWithFilter / readContent") getLocals
            if retry = 0
            then return! loop (retry + 1)
            else return None
    }
    loop 0

## deleteDirectoryAsync

In [ ]:
let inline deleteDirectoryAsync path =
    let rec loop retry = async {
        try
            System.IO.Directory.Delete (path, true)
            return retry
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path |> System.IO.Path.GetFileName} / ex: {ex |> printException} / {getLocals ()}"
                trace Debug (fun () -> "deleteDirectoryAsync") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test"

let inline lockDirectory () = async {
    trace Debug (fun () -> "_1") getLocals
    System.IO.File.WriteAllText (path </> "test.txt", "0")
    use _ = new System.IO.FileStream (
        path </> "test.txt",
        System.IO.FileMode.Open,
        System.IO.FileAccess.ReadWrite,
        System.IO.FileShare.None
    )
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    Directory.CreateDirectory path |> ignore
    trace Debug (fun () -> "2") getLocals
    let! child = lockDirectory () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 60
    trace Debug (fun () -> "4") getLocals
    let! retries = deleteDirectoryAsync path
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries
}
|> Async.runWithTimeout 3000
|> function
    | Some retries ->
        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 0)
            (if Runtime.isWindows () then 150 else 0)

        true
    | _ -> false
|> _equal true

00:00:02 #16 [Debug] 1
00:00:02 #17 [Debug] 2
00:00:02 #18 [Debug] 3
00:00:02 #19 [Debug] _1
00:00:02 #20 [Debug] _2
00:00:02 #21 [Debug] 4
00:00:02 #22 [Debug] deleteDirectoryAsync / path: test / ex: System.IO.IOException: The process cannot access the file 'test.txt' because it is being used by another process.
00:00:04 #23 [Debug] deleteDirectoryAsync / path: test / ex: System.IO.IOException: The process cannot access the file 'test.txt' because it is being used by another process.
00:00:04 #24 [Debug] _3
00:00:04 #25 [Debug] 5
00:00:04 #26 [Debug] 6
124
124
124
True


## deleteFileAsync

In [ ]:
let inline deleteFileAsync path =
    let rec loop retry = async {
        try
            System.IO.File.Delete path
            return retry
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path |> System.IO.Path.GetFileName} / ex: {ex |> printException} / {getLocals ()}"
                trace Warning (fun () -> "deleteFileAsync") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"

let inline lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use _ = new System.IO.FileStream (
        path,
        System.IO.FileMode.Open,
        System.IO.FileAccess.ReadWrite,
        System.IO.FileShare.None
    )
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! "0" |> writeAllTextAsync path
    trace Debug (fun () -> "2") getLocals
    let! child = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    let! retries = deleteFileAsync path
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries
}
|> Async.runWithTimeout 3000
|> function
    | Some retries ->
        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 0)
            (if Runtime.isWindows () then 150 else 0)

        true
    | _ -> false
|> _equal true

00:00:05 #27 [Debug] 1
00:00:05 #28 [Debug] 2
00:00:05 #29 [Debug] 3
00:00:05 #30 [Debug] _1
00:00:05 #31 [Debug] _2
00:00:05 #32 [Debug] 4
00:00:05 #33 [Warning] deleteFileAsync / path: test.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231027-1826-3086-8667-8d50f7b4aa46\test.txt' because it is being used by another process.
00:00:06 #34 [Warning] deleteFileAsync / path: test.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231027-1826-3086-8667-8d50f7b4aa46\test.txt' because it is being used by another process.
00:00:07 #35 [Debug] _3
00:00:07 #36 [Debug] 5
00:00:07 #37 [Debug] 6
128
128
128
True


## moveFileAsync

In [ ]:
let inline moveFileAsync newPath oldPath =
    let rec loop retry = async {
        try
            System.IO.File.Move (oldPath, newPath)
            return retry
        with ex ->
            if retry % 100 = 0 then
                let getLocals () =
                    $"oldPath: {oldPath |> System.IO.Path.GetFileName} / newPath: {newPath |> System.IO.Path.GetFileName} / ex: {ex |> printException} / {getLocals ()}"
                trace Warning (fun () -> "moveFileAsync") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"
let newPath = tempFolder </> "test2.txt"

let inline lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use _ = new System.IO.FileStream (
        path,
        System.IO.FileMode.Open,
        System.IO.FileAccess.ReadWrite,
        System.IO.FileShare.None
    )
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! "0" |> writeAllTextAsync path
    trace Debug (fun () -> "2") getLocals
    let! child = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    let! retries1 = path |> moveFileAsync newPath
    trace Debug (fun () -> "5") getLocals
    let! retries2 = newPath |> waitForFileAccess None
    trace Debug (fun () -> "6") getLocals
    let! text = newPath |> readAllTextAsync
    trace Debug (fun () -> "7") getLocals
    do! child
    trace Debug (fun () -> "8") getLocals
    return retries1, retries2, text
}
|> Async.runWithTimeout 3000
|> function
    | Some (retries1, retries2, text) ->
        retries1
        |> _isBetween
            (if Runtime.isWindows () then 100 else 0)
            (if Runtime.isWindows () then 150 else 0)

        retries2
        |> _isBetween
            (if Runtime.isWindows () then 0 else 100)
            (if Runtime.isWindows () then 0 else 200)
        
        text |> _equal "0"
        
        true
    | _ -> false
|> _equal true

00:00:07 #38 [Debug] 1
00:00:07 #39 [Debug] 2
00:00:07 #40 [Debug] 3
00:00:07 #41 [Debug] _1
00:00:07 #42 [Debug] _2
00:00:07 #43 [Debug] 4
00:00:07 #44 [Warning] moveFileAsync / oldPath: test.txt / newPath: test2.txt / ex: System.IO.IOException: The process cannot access the file because it is being used by another process.
00:00:09 #45 [Warning] moveFileAsync / oldPath: test.txt / newPath: test2.txt / ex: System.IO.IOException: The process cannot access the file because it is being used by another process.
00:00:09 #46 [Debug] _3
00:00:09 #47 [Debug] 5
00:00:09 #48 [Debug] 6
00:00:09 #49 [Debug] 7
00:00:09 #50 [Debug] 8
128
128
128
0
0
0
0
True


## watchDirectory

In [ ]:
[<RequireQualifiedAccess>]
type FileSystemChangeType =
    | Failure
    | Changed
    | Created
    | Deleted
    | Renamed

[<RequireQualifiedAccess>]
type FileSystemChange =
    | Failure of exn: exn
    | Changed of path: string * content: string option
    | Created of path: string * content: string option
    | Deleted of path: string
    | Renamed of oldPath: string * (string * string option)


let inline watchDirectoryWithFilter filter shouldReadContent path =
    let fullPath = path |> System.IO.Path.GetFullPath
    let getLocals () = $"filter: {filter} / {getLocals ()}"

    let watcher =
        new System.IO.FileSystemWatcher (
            Path = fullPath,
            NotifyFilter = filter,
            EnableRaisingEvents = true,
            IncludeSubdirectories = true
        )

    let inline getEventPath (path : string) =
        path |> String.trim |> String.replace fullPath "" |> String.trimStart [| '/'; '\\' |]

    let inline ticks () =
        System.DateTime.UtcNow.Ticks

    let changedStream =
        AsyncSeq.subscribeEvent
            watcher.Changed
            (fun event ->
                ticks (),
                [ FileSystemChange.Changed (getEventPath event.FullPath, None) ]
            )

    let deletedStream =
        AsyncSeq.subscribeEvent
            watcher.Deleted
            (fun event ->
                ticks (),
                [ FileSystemChange.Deleted (getEventPath event.FullPath) ]
            )

    let createdStream =
        AsyncSeq.subscribeEvent
            watcher.Created
            (fun event ->
                let path = getEventPath event.FullPath
                ticks (), [
                    FileSystemChange.Created (path, None)
                    if Runtime.isWindows () then
                        FileSystemChange.Changed (path, None)
                ])

    let renamedStream =
        AsyncSeq.subscribeEvent
            watcher.Renamed
            (fun event ->
                ticks (), [
                    FileSystemChange.Renamed (
                        getEventPath event.OldFullPath,
                        (getEventPath event.FullPath, None)
                    )
                ]
            )

    let failureStream =
        AsyncSeq.subscribeEvent
            watcher.Error
            (fun event -> ticks (), [ FileSystemChange.Failure (event.GetException ()) ])

    let stream =
        [
            changedStream
            deletedStream
            createdStream
            renamedStream
            failureStream
        ]
        |> FSharp.Control.AsyncSeq.mergeAll
        |> FSharp.Control.AsyncSeq.map (fun (t, events) ->
            events
            |> List.fold
                (fun (i, events) event ->
                    i + 1L,
                    (t + i, event) :: events)
                (0L, [])
            |> snd
            |> List.rev
        )
        |> FSharp.Control.AsyncSeq.concatSeq
        |> FSharp.Control.AsyncSeq.mapAsyncParallel (fun (t, event) -> async {
            match shouldReadContent event, event with
            | true, FileSystemChange.Changed (path, _) ->
                do! Async.Sleep 5
                let! content = fullPath </> path |> readAllTextRetryAsync
                return t, FileSystemChange.Changed (path, content)
            | true, FileSystemChange.Created (path, _) ->
                do! Async.Sleep 5
                let! content = fullPath </> path |> readAllTextRetryAsync
                return t, FileSystemChange.Created (path, content)
            | true, FileSystemChange.Renamed (oldPath, (newPath, _)) ->
                let! content = fullPath </> newPath |> readAllTextRetryAsync
                return t, FileSystemChange.Renamed (oldPath, (newPath, content))
            | _ -> return t, event
        })

    let disposable =
        newDisposable (fun () ->
            trace Debug (fun () -> "watchWithFilter / Disposing watch stream") getLocals
            watcher.EnableRaisingEvents <- false
            watcher.Dispose ()
        )

    stream, disposable

let inline watchDirectory path =
    watchDirectoryWithFilter
        (System.IO.NotifyFilters.FileName
        // ||| System.IO.NotifyFilters.DirectoryName
        // ||| System.IO.NotifyFilters.Attributes
        //// ||| System.IO.NotifyFilters.Size
        ||| System.IO.NotifyFilters.LastWrite
        //// ||| System.IO.NotifyFilters.LastAccess
        // ||| System.IO.NotifyFilters.CreationTime
        // ||| System.IO.NotifyFilters.Security
        )
        path

### testEventsRaw (test)

In [ ]:
//// test

let inline testEventsRaw
    (watchFn : (_ -> bool) -> string -> FSharp.Control.AsyncSeq<int64 * FileSystemChange> * IDisposable)
    write
    =
    let tempDirectory = createTempDirectory ()
    let stream, disposable = watchFn (fun _ -> true) tempDirectory

    let events = System.Collections.Concurrent.ConcurrentBag ()

    let inline iter () =
        stream
        |> FSharp.Control.AsyncSeq.iterAsyncParallel (fun event -> async { events.Add event })

    let run = async {
        let! _ = iter () |> Async.StartChild
        do! Async.Sleep 250
        return! write tempDirectory
    }

    try
        run
        |> Async.runWithTimeout 60000
        |> _equal (Some ())
    finally
        disposable.Dispose ()
        deleteDirectoryAsync tempDirectory |> Async.Ignore |> Async.RunSynchronously

    let eventsLog =
        events
        |> Seq.toList
        |> List.sortBy fst
        |> List.fold
            (fun (prev, acc) (ticks, event) ->
                ticks, (ticks, (if prev = 0L then 0L else ticks - prev), event) :: acc
            )
            (0L, [])
        |> snd
        |> List.rev
        |> List.map (fun (diff, n, event) ->
            let text = $"{n} / {diff} / {event}"
            if text |> String.length <= 100
            then text
            else text |> String.substring 0 100 |> String.replace "\n" ""
        )
        |> String.concat "\n"
    let getLocals () = $"eventsLog: \n{eventsLog} / {getLocals ()}"
    trace Debug (fun () -> "testEventsRaw") getLocals

    events
    |> Seq.toList
    |> List.sortBy fst
    |> List.map snd
    |> List.fold
        (fun acc event ->
            match acc, event with
            | FileSystemChange.Changed (lastPath, Some lastContent) as lastEvent :: acc,
                FileSystemChange.Changed (path, Some content)
                when lastPath = path && content |> String.startsWith lastContent
                ->
                event :: acc
            | _ -> event :: acc
        )
        []
    |> List.rev

#### fast (test)

In [ ]:
//// test

let inline write path = async {
    let n = 2

    for i = 1 to n do
        do! $"a{i}" |> writeAllTextAsync (path </> $"file{i}.txt")

    do! Async.Sleep 250

    for i = 1 to n do
        do! $"b{i}" |> writeAllTextAsync (path </> $"file{i}.txt")

    do! Async.Sleep 250

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    do! Async.Sleep 250

    for i = 1 to n do
        do! $"c{i}" |> writeAllTextAsync (path </> $"file_{i}.txt")

    do! Async.Sleep 250

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    do! Async.Sleep 250
}

let inline run () =
    let events = testEventsRaw watchDirectory write

    events
    |> _sequenceEqual [
        FileSystemChange.Created ("file1.txt", Some "a1")
        FileSystemChange.Changed ("file1.txt", Some "a1")
        FileSystemChange.Created ("file2.txt", Some "a2")
        FileSystemChange.Changed ("file2.txt", Some "a2")

        FileSystemChange.Changed ("file1.txt", Some "b1")
        FileSystemChange.Changed ("file2.txt", Some "b2")

        FileSystemChange.Renamed ("file1.txt", ("file_1.txt", Some "b1"))
        FileSystemChange.Renamed ("file2.txt", ("file_2.txt", Some "b2"))

        FileSystemChange.Changed ("file_1.txt", Some "c1")
        FileSystemChange.Changed ("file_2.txt", Some "c2")

        FileSystemChange.Deleted "file_1.txt"
        FileSystemChange.Deleted "file_2.txt"
    ]

run
|> retryFn 3
|> _equal (Some ())

FSharpOption<Unit>
      Value: <null>
00:00:16 #51 [Debug] watchWithFilter / Disposing watch stream / filter: FileName, LastWrite
00:00:16 #52 [Debug] testEventsRaw / eventsLog: 
0 / 638340424008216707 / Created ("file1.txt", Some "a1")
1 / 638340424008216708 / Changed ("file1.txt", Some "a1")
34298 / 638340424008251006 / Changed ("file1.txt", Some "a1")
1180 / 638340424008252186 / Created ("file2.txt", Some "a2")
1 / 638340424008252187 / Changed ("file2.txt", Some "a2")
95 / 638340424008252282 / Changed ("file2.txt", Some "a2")
2403744 / 638340424010656026 / Changed ("file1.txt", Some "b1")
2680 / 638340424010658706 / Changed ("file1.txt", Some "b1")
12280 / 638340424010670986 / Changed ("file2.txt", Some "b2")
2053 / 638340424010673039 / Changed ("file2.txt", Some "b2")
2603772 / 638340424013276811 / Renamed ("file1.txt", ("file_1.txt", Some "b1"))
7087 / 638340424013283898 / Renamed ("file2.txt", ("file_2.txt", Some "b2"))
2524141 / 638340424015808039 / Changed ("file_1.txt", Some 

#### slow (test)

In [ ]:
//// test

let inline write path = async {
    let n = 2

    let contents =
        [ 1 .. n ]
        |> List.map (string >> String.replicate 1_000_000)

    for i = 1 to n do
        do! $"{contents.[i - 1]}a" |> writeAllTextAsync (path </> $"file{i}.txt")

    do! Async.Sleep 1500

    for i = 1 to n do
        do! $"{contents.[i - 1]}b" |> writeAllTextAsync (path </> $"file{i}.txt")

    do! Async.Sleep 1500

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    do! Async.Sleep 1500

    for i = 1 to n do
        do! $"{contents.[i - 1]}c" |> writeAllTextAsync (path </> $"file_{i}.txt")

    do! Async.Sleep 1500

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    do! Async.Sleep 1500
}

let inline run () =
    let events =
        testEventsRaw watchDirectory write
        |> List.map (function
            | FileSystemChange.Changed (path, Some content) ->
                FileSystemChange.Changed (path, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
            | FileSystemChange.Created (path, Some content) ->
                FileSystemChange.Created (path, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
            | FileSystemChange.Renamed (oldPath, (newPath, Some content)) ->
                FileSystemChange.Renamed (
                    oldPath,
                    (newPath, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
                )
            | event -> event
        )

    events
    |> _sequenceEqual [
        FileSystemChange.Created ("file1.txt", Some "1a")
        FileSystemChange.Changed ("file1.txt", Some "1a")
        FileSystemChange.Created ("file2.txt", Some "2a")
        FileSystemChange.Changed ("file2.txt", Some "2a")

        FileSystemChange.Changed ("file1.txt", Some "1b")
        FileSystemChange.Changed ("file2.txt", Some "2b")

        FileSystemChange.Renamed ("file1.txt", ("file_1.txt", Some "1b"))
        FileSystemChange.Renamed ("file2.txt", ("file_2.txt", Some "2b"))

        FileSystemChange.Changed ("file_1.txt", Some "1c")
        FileSystemChange.Changed ("file_2.txt", Some "2c")

        FileSystemChange.Deleted "file_1.txt"
        FileSystemChange.Deleted "file_2.txt"
    ]

run
|> retryFn 5
|> _equal (Some ())

00:00:22 #53 [Debug] watchWithFilter / readContent / retry: 0 / ex: System.AggregateException: One or more errors occurred. (The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231027-1826-4643-4350-4d77f6f20a1b\file1.txt' because it is being used by another process.)
00:00:22 #54 [Debug] watchWithFilter / readContent / retry: 0 / ex: System.AggregateException: One or more errors occurred. (The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231027-1826-4643-4350-4d77f6f20a1b\file2.txt' because it is being used by another process.)
00:00:22 #55 [Debug] waitForFileAccess / path: file2.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20231027-1826-4643-4350-4d77f6f20a1b\file2.txt' because it is being used by another process.
00:00:25 #56 [Debug] watchWithFilter / readContent / retry: 0 / ex: System.AggregateException: One or more errors occurred. (The proce

### testEventsSorted (test)

In [ ]:
//// test

let inline sortEvent event =
    match event with
    | FileSystemChange.Failure _ -> 0
    | FileSystemChange.Created _ -> 1
    | FileSystemChange.Changed _ -> 2
    | FileSystemChange.Renamed (_oldPath, _) -> 3
    | FileSystemChange.Deleted _ -> 4

let inline formatEvents events =
    events
    |> Seq.toList
    |> List.sortBy (snd >> sortEvent)
    |> List.choose (fun (ticks, event) ->
        match event with
        | FileSystemChange.Failure _ ->
            None
        | FileSystemChange.Changed (path, _) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Changed)
        | FileSystemChange.Created (path, _) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Created)
        | FileSystemChange.Deleted path ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Deleted)
        | FileSystemChange.Renamed (_oldPath, (path, _)) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Renamed)
    )
    |> List.sortBy (fun (_, path, _) -> path)
    |> List.distinctBy (fun (_, path, event) -> path, event)

let inline testEventsSorted
    (watchFn : string -> FSharp.Control.AsyncSeq<int64 * FileSystemChange> * IDisposable)
    write
    =
    let path = createTempDirectory ()
    let stream, disposable = watchFn path

    let events = System.Collections.Concurrent.ConcurrentBag ()

    let inline iter () =
        stream
        |> FSharp.Control.AsyncSeq.iterAsyncParallel (fun event -> async { events.Add event })

    let run = async {
        let! _ = iter () |> Async.StartChild
        do! Async.Sleep 250
        return! write path
    }

    try
        run
        |> Async.runWithTimeout 5000
        |> _equal (Some ())
    finally
        disposable.Dispose ()
        deleteDirectoryAsync path |> Async.Ignore |> Async.RunSynchronously

    let events = formatEvents events

    let eventMap =
        events
        |> List.map (fun (ticks, path, event) -> path, (event, ticks))
        |> List.groupBy fst
        |> List.map (fun (path, events) ->
            let event, _ticks =
                events
                |> List.map snd
                |> List.sortByDescending snd
                |> List.head

            path, event
        )
        |> Map.ofList

    let eventList =
        events
        |> List.map (fun (_ticks, path, event) -> path, event)

    eventMap, eventList

#### create and delete (test)

In [ ]:
//// test

let inline write path = async {
    let n = 3

    for i = 1 to n do
        do! $"{i}" |> writeAllTextAsync (path </> $"file{i}.txt")

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file{i}.txt") |> Async.Ignore
    
    do! Async.Sleep 150
}

let inline run () =
    let eventMap, eventList = testEventsSorted (watchDirectory (fun _ -> false)) write

    [
        "file1.txt", nameof FileSystemChangeType.Created
        "file1.txt", nameof FileSystemChangeType.Changed
        "file1.txt", nameof FileSystemChangeType.Deleted

        "file2.txt", nameof FileSystemChangeType.Created
        "file2.txt", nameof FileSystemChangeType.Changed
        "file2.txt", nameof FileSystemChangeType.Deleted

        "file3.txt", nameof FileSystemChangeType.Created
        "file3.txt", nameof FileSystemChangeType.Changed
        "file3.txt", nameof FileSystemChangeType.Deleted
    ]
    |> _sequenceEqual eventList

    [
        "file1.txt", nameof FileSystemChangeType.Deleted
        "file2.txt", nameof FileSystemChangeType.Deleted
        "file3.txt", nameof FileSystemChangeType.Deleted
    ]
    |> Map.ofList
    |> _sequenceEqual eventMap

run
|> retryFn 3
|> _equal (Some ())

FSharpOption<Unit>
      Value: <null>
00:00:32 #62 [Debug] watchWithFilter / Disposing watch stream / filter: FileName, LastWrite
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file1.txt, Deleted )
( file2.txt, Created )
( file2.txt, Changed )
( file2.txt, Deleted )
( file3.txt, Created )
( file3.txt, Changed )
( file3.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Deleted
      - Key: file2.txt
        Value: Deleted
      - Key: file3.txt
        Value: Deleted
FSharpOption<Unit>
      Value: <null>


#### change (test)

In [ ]:
//// test

let inline write path = async {
    let n = 2

    for i = 1 to n do
        do! $"{i}" |> writeAllTextAsync (path </> $"file{i}.txt")

    for i = 1 to n do
        do! "" |> writeAllTextAsync (path </> $"file{i}.txt")

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file{i}.txt") |> Async.Ignore
    
    do! Async.Sleep 150
}

let inline run () =
    let eventMap, eventList = testEventsSorted (watchDirectory (fun _ -> false)) write

    [
        "file1.txt", nameof FileSystemChangeType.Created
        "file1.txt", nameof FileSystemChangeType.Changed
        "file1.txt", nameof FileSystemChangeType.Deleted

        "file2.txt", nameof FileSystemChangeType.Created
        "file2.txt", nameof FileSystemChangeType.Changed
        "file2.txt", nameof FileSystemChangeType.Deleted
    ]
    |> _sequenceEqual eventList

    [
        "file1.txt", nameof FileSystemChangeType.Deleted
        "file2.txt", nameof FileSystemChangeType.Deleted
    ]
    |> Map.ofList
    |> _sequenceEqual eventMap

run
|> retryFn 3
|> _equal (Some ())

FSharpOption<Unit>
      Value: <null>
00:00:37 #63 [Debug] watchWithFilter / Disposing watch stream / filter: FileName, LastWrite
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file1.txt, Deleted )
( file2.txt, Created )
( file2.txt, Changed )
( file2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Deleted
      - Key: file2.txt
        Value: Deleted
FSharpOption<Unit>
      Value: <null>


#### rename (test)

In [ ]:
//// test

let inline write path = async {
    let n = 2

    for i = 1 to n do
        do! $"{i}" |> writeAllTextAsync (path </> $"file{i}.txt")

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt") |> Async.Ignore
    
    do! Async.Sleep 150
}

let inline run () =
    let eventMap, eventList = testEventsSorted (watchDirectory (fun _ -> false)) write

    [
        "file1.txt", nameof FileSystemChangeType.Created
        "file1.txt", nameof FileSystemChangeType.Changed
        "file2.txt", nameof FileSystemChangeType.Created
        "file2.txt", nameof FileSystemChangeType.Changed

        "file_1.txt", nameof FileSystemChangeType.Renamed
        "file_1.txt", nameof FileSystemChangeType.Deleted

        "file_2.txt", nameof FileSystemChangeType.Renamed
        "file_2.txt", nameof FileSystemChangeType.Deleted
    ]
    |> _sequenceEqual eventList

    [
        "file1.txt", nameof FileSystemChangeType.Changed
        "file2.txt", nameof FileSystemChangeType.Changed
        "file_1.txt", nameof FileSystemChangeType.Deleted
        "file_2.txt", nameof FileSystemChangeType.Deleted
    ]
    |> Map.ofList
    |> _sequenceEqual eventMap

run
|> retryFn 3
|> _equal (Some ())

FSharpOption<Unit>
      Value: <null>
00:00:41 #64 [Debug] watchWithFilter / Disposing watch stream / filter: FileName, LastWrite
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file2.txt, Created )
( file2.txt, Changed )
( file_1.txt, Renamed )
( file_1.txt, Deleted )
( file_2.txt, Renamed )
( file_2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Changed
      - Key: file2.txt
        Value: Changed
      - Key: file_1.txt
        Value: Deleted
      - Key: file_2.txt
        Value: Deleted
FSharpOption<Unit>
      Value: <null>


#### full (test)

In [ ]:
//// test

let inline write path = async {
    let n = 2

    for i = 1 to n do
        do! $"{i}" |> writeAllTextAsync (path </> $"file{i}.txt")

    for i = 1 to n do
        do! "" |> writeAllTextAsync (path </> $"file{i}.txt")

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    for i = 1 to n do
        do! $"{i}" |> writeAllTextAsync (path </> $"file_{i}.txt")

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt") |> Async.Ignore
    
    do! Async.Sleep 150
}

let inline run () =
    let eventMap, eventList = testEventsSorted (watchDirectory (fun _ -> false)) write

    [
        "file1.txt", nameof FileSystemChangeType.Created
        "file1.txt", nameof FileSystemChangeType.Changed
        "file2.txt", nameof FileSystemChangeType.Created
        "file2.txt", nameof FileSystemChangeType.Changed

        "file_1.txt", nameof FileSystemChangeType.Changed
        "file_1.txt", nameof FileSystemChangeType.Renamed
        "file_1.txt", nameof FileSystemChangeType.Deleted

        "file_2.txt", nameof FileSystemChangeType.Changed
        "file_2.txt", nameof FileSystemChangeType.Renamed
        "file_2.txt", nameof FileSystemChangeType.Deleted
    ]
    |> _sequenceEqual eventList

    [
        "file1.txt", nameof FileSystemChangeType.Changed
        "file2.txt", nameof FileSystemChangeType.Changed
        "file_1.txt", nameof FileSystemChangeType.Deleted
        "file_2.txt", nameof FileSystemChangeType.Deleted
    ]
    |> Map.ofList
    |> _sequenceEqual eventMap

run
|> retryFn 3
|> _equal (Some ())

FSharpOption<Unit>
      Value: <null>
00:00:47 #65 [Debug] watchWithFilter / Disposing watch stream / filter: FileName, LastWrite
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file2.txt, Created )
( file2.txt, Changed )
( file_1.txt, Changed )
( file_1.txt, Renamed )
( file_1.txt, Deleted )
( file_2.txt, Changed )
( file_2.txt, Renamed )
( file_2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Changed
      - Key: file2.txt
        Value: Changed
      - Key: file_1.txt
        Value: Deleted
      - Key: file_2.txt
        Value: Deleted
FSharpOption<Unit>
      Value: <null>
